In [1]:
import pandas as pd
import bs4 as bs
import urllib.request
from tqdm import tqdm
import numpy as np

# load story metadata

In [2]:
url_root = 'https://www.allsides.com/'
url_page = url_root + 'story/admin?page='
tab_num = 0

# loop over pages
# there are about 100 pages as of 12/29/2020, so we set this number to be well above that
dfs = []
for tab_num in tqdm(range(150)):
    # read in the source
    try:
        source = urllib.request.urlopen(url_page + str(tab_num))
        sp = bs.BeautifulSoup(source, 'lxml')
        table = sp.table
        df = pd.read_html(str(table), encoding='utf-8', header=0)[0] # read table with no links

        # get links
        links = []
        for tag in table.find_all('a'):
            if tag.has_attr('href'):
                link = tag.get('href')
                if '/story' in link:
                    links.append(link)
            else:
                print(f'error! missing a link for {link}')
        df['url_story'] = links
        dfs.append(df)
    except:
        print(f'stopped after {tab_num} pages')
        break
        
df = pd.concat(dfs)
df.to_pickle('../data/df_links.pkl')

 71%|███████▏  | 107/150 [01:34<00:39,  1.09it/s]/Users/chandan/Library/Python/3.7/lib/python/site-packages/bs4/__init__.py:177: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
 72%|███████▏  | 108/150 [01:35<00:35,  1.18it/s]

stopped after 107 pages


 73%|███████▎  | 109/150 [01:36<00:30,  1.36it/s]

stopped after 108 pages


 73%|███████▎  | 110/150 [01:36<00:28,  1.40it/s]

...

stopped after 146 pages
stopped after 147 pages


 99%|█████████▉| 149/150 [01:48<00:00,  6.13it/s]

stopped after 148 pages


100%|██████████| 150/150 [01:49<00:00,  3.25it/s]

stopped after 149 pages


In [10]:
# add info from link
def get_info_from_url_story(url_story):
    '''add info rom url on a a story
    '''
    story = urllib.request.urlopen(url_story)
    sp_story = bs.BeautifulSoup(story, 'html.parser')

    
    # extract info from specific story page
    story_triplet_info = {}
    
    # loop over left, center, and right stories
    try:
        for div in sp_story.find_all('div', {'class': 'news-title'})[:3]:
            title = div.a.contents[0]
            url = div.a.get('href')

            news_source = div.parent.find_all('div', {'class': 'news-source'})[0].contents[1]
            leaning = div.parent.find_all('div', {'class': 'bias-image'})[0].img.get('title').replace("Political News Media Bias Rating: ", '')

            news_text = '\n'.join([s.contents[0] for s in 
                      div.parent.find_all('div', {'class': 'news-body'})[0].contents
                      if 'Tag' in str(type(s))])
            prefix = leaning.lower().replace('lean ', '') + '_story_'
            story_info = {
                f'{prefix}title': title,
                f'{prefix}url': url,
                f'{prefix}source': news_source,
                f'{prefix}leaning': leaning,
                f'{prefix}text': news_text
            }
            story_triplet_info = {**story_triplet_info, **story_info}
    except:
        return {}
    return story_triplet_info

def get_stories(df):
    '''Add list for all stories
    '''
    story_triplet_list = []
    for i in tqdm(range(df.shape[0])):
        url_story = url_root + df.iloc[i]['url_story']
        story_triplet_info = get_info_from_url_story(url_story)
        story_triplet_list.append(story_triplet_info)
    return pd.DataFrame.from_dict(story_triplet_list)
    
df_stories = get_stories(df)

# return df.join(df_stories).dropna()

100%|██████████| 5315/5315 [1:34:55<00:00,  1.02s/it]  


In [11]:
df_stories.to_pickle('../data/df_stories.pkl')
# df_full.head()

RecursionError: maximum recursion depth exceeded while getting the str of an object

In [13]:
df_stories.shape

(5315, 25)

In [16]:
pkl.dump(df_stories, open('test.pkl', 'wb'))

RecursionError: maximum recursion depth exceeded while pickling an object

In [15]:
import pickle as pkl

In [17]:
from copy import deepcopy

In [18]:
dd = deepcopy(df_stories)

In [20]:
dd.shape

(5315, 25)

In [23]:
dd.head()

,right_story_title,right_story_url,right_story_source,right_story_leaning,right_story_text,center_story_title,center_story_url,center_story_source,center_story_leaning,center_story_text,...,mixed_story_title,mixed_story_url,mixed_story_source,mixed_story_leaning,mixed_story_text,not rated_story_title,not rated_story_url,not rated_story_source,not rated_story_leaning,not rated_story_text
0,COVID-19 Relief Checks To Hit Some Bank Accoun...,https://dailycaller.com/2020/12/29/covid-19-re...,The Daily Caller,Right,The U.S. Treasury Secretary Steve Mnuchin said...,You could receive your $600 stimulus check as ...,https://www.usatoday.com/story/money/2020/12/2...,USA TODAY,Center,Stimulus money could be coming to your account...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Red states prioritize elderly and vulnerable i...,https://www.washingtonexaminer.com/news/red-st...,Washington Examiner,Lean Right,Several red states are eschewing federal recom...,First patient to receive COVID-19 vaccine gets...,https://thehill.com/news-by-subject/healthcare...,The Hill,Center,"Margaret Keenan, the first person outside of t...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Trump Dethrones Obama for 'Most Admired Man' i...,https://townhall.com/tipsheet/cortneyobrien/20...,Townhall,Right,Former President Obama's 12-year reign is over...,"Donald Trump, Michelle Obama Most Admired in 2020",https://news.gallup.com/poll/328193/donald-tru...,Gallup,Center,DATA\nAmericans are most likely to name Presid...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Marco Rubio backs $2,000 stimulus checks, call...",https://www.foxnews.com/politics/marco-rubio-2...,Fox News (Online News),Lean Right,Republican Florida Sen. Marco Rubio on Monday ...,"U.S. Senate wrangles over $2,000 stimulus chec...",https://www.reuters.com/article/us-usa-trump/u...,Reuters,Center,The U.S. Senate will grapple on Tuesday with w...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,New York Times accused of 'glorifying' cancel ...,https://www.foxnews.com/media/new-york-times-c...,Fox News (Online News),Lean Right,"The New York Times was accused on Sunday of ""g...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
for k in dd:
    dd[k] = dd[k].astype(str)

In [29]:
dd.to_pickle('../data/df_stories.pkl')